In [4]:
import pandas as pd

In [5]:
# Assuming the CSV file is named 'stock_data.csv'
df = pd.read_csv('combined_stock_data.csv')

In [6]:
# Ensure the Date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort the data by Date for each Symbol
df = df.sort_values(by=['Symbol', 'Date'])

# Calculate the daily returns
df['Daily_Return'] = df.groupby('Symbol')['Adj Close'].pct_change()

In [8]:
df.set_index('Date', inplace=True)


In [9]:
df

In [10]:
returns_df = df.pivot(columns='Symbol', values='Daily_Return')

In [11]:
returns_df

In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv('combined_stock_data.csv')

# Ensure the 'Date' column is a datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Add a column for year and month
df['YearMonth'] = df['Date'].dt.to_period('M')

# Calculate daily returns
df['Daily_Return'] = df.groupby('Symbol')['Adj Close'].pct_change()

# Calculate monthly returns by compounding daily returns
df['Monthly_Return'] = df.groupby(['Symbol', 'YearMonth'])['Daily_Return'].apply(lambda x: (1 + x).prod() - 1)

# Calculate average daily turnover
df['Daily_Turnover'] = df['Volume'] / df['Outstanding_Shares']
df['Monthly_Avg_Turnover'] = df.groupby(['Symbol', 'YearMonth'])['Daily_Turnover'].transform('mean')

# Aggregate monthly data
monthly_df = df.groupby(['Symbol', 'YearMonth']).agg({
    'Monthly_Return': 'last',
    'Monthly_Avg_Turnover': 'last'
}).reset_index()

# Define the ranking period (J months) and holding period (K months)
J = 3
K = 1

# Rank stocks based on past J-month returns and turnover
monthly_df['Rank_Return'] = monthly_df.groupby('YearMonth')['Monthly_Return'].rank(method='first')
monthly_df['Rank_Turnover'] = monthly_df.groupby('YearMonth')['Monthly_Avg_Turnover'].rank(method='first')

# Define the number of return and turnover portfolios
num_return_portfolios = 5
num_turnover_portfolios = 3

# Assign stocks to return portfolios
monthly_df['Return_Portfolio'] = monthly_df.groupby('YearMonth')['Rank_Return'].apply(lambda x: pd.qcut(x, num_return_portfolios, labels=False))

# Assign stocks to turnover portfolios
monthly_df['Turnover_Portfolio'] = monthly_df.groupby('YearMonth')['Rank_Turnover'].apply(lambda x: pd.qcut(x, num_turnover_portfolios, labels=False))

# Combine return and turnover portfolios to create the final portfolios
monthly_df['Final_Portfolio'] = monthly_df['Return_Portfolio'].astype(str) + '-' + monthly_df['Turnover_Portfolio'].astype(str)

# Calculate portfolio metrics (e.g., average return) for each holding period
portfolio_returns = monthly_df.groupby(['YearMonth', 'Final_Portfolio']).agg({
    'Monthly_Return': 'mean',
    'Monthly_Avg_Turnover': 'mean'
}).reset_index()

# Display the results
print(portfolio_returns)
